In [2]:
import os
import sys
import numpy as np
import networkx as nx
from rdkit import Chem

# ================= 0. 环境修复 =================
# 解决 NetworkX 3.x 兼容性
if not hasattr(nx, 'from_numpy_matrix'):
    print("正在应用 NetworkX 版本热修复...")
    nx.from_numpy_matrix = nx.from_numpy_array

# ================= 1. 定义 SMILES 预处理类 =================
class SmilesPreprocessor(object):
    def __init__(self, add_Hs=False, kekulize=True, max_atoms=48, max_size=48):
        self.add_Hs = add_Hs
        self.kekulize = kekulize
        self.max_atoms = max_atoms
        self.max_size = max_size

    def _prepare_mol(self, Smiles):
        mol = Chem.MolFromSmiles(Smiles)
        if mol is None: return None, None
        canonical_smiles = Chem.MolToSmiles(mol)
        if self.add_Hs:
            mol = Chem.AddHs(mol)
        if self.kekulize:
            Chem.Kekulize(mol, clearAromaticFlags=True)
        return mol, canonical_smiles

    def _get_features(self, mol):
        # 检查原子数量
        if self.max_atoms >= 0 and mol.GetNumAtoms() > self.max_atoms:
            return None, None, None
            
        # 构建原子特征 (Node Features)
        atom_list = [a.GetAtomicNum() for a in mol.GetAtoms()]
        n_atom = len(atom_list)
        atom_array = np.zeros(self.max_size, dtype=np.int32)
        atom_array[:n_atom] = np.array(atom_list, dtype=np.int32)
        
        # 构建边特征 (Adj Features)
        adj_array = np.zeros((4, self.max_size, self.max_size), dtype=np.float32)
        bond_type_to_channel = {
            Chem.BondType.SINGLE: 0, Chem.BondType.DOUBLE: 1,
            Chem.BondType.TRIPLE: 2, Chem.BondType.AROMATIC: 3
        }
        for bond in mol.GetBonds():
            bt = bond.GetBondType()
            if bt in bond_type_to_channel:
                ch = bond_type_to_channel[bt]
                i = bond.GetBeginAtomIdx()
                j = bond.GetEndAtomIdx()
                adj_array[ch, i, j] = 1.0
                adj_array[ch, j, i] = 1.0
                
        return atom_array, adj_array, mol.GetNumAtoms()

    def process(self, smiles):
        try:
            mol, canonical_smiles = self._prepare_mol(smiles)
            if mol is None: return None, None, None, None
            atom_array, adj_array, mol_size = self._get_features(mol)
            return atom_array, adj_array, mol_size, canonical_smiles
        except:
            return None, None, None, None

# ================= 2. 定义 Dataset3 处理器 =================
class Dataset3_Processor(object):
    def __init__(self, in_path, out_path, freedom=0, max_samples=None): # [修改] 默认值为 None，表示无限制
        self.in_path = in_path
        self.out_path = out_path
        self.freedom = freedom
        self.max_samples = max_samples 

        # [修改] 完整的原子列表 (包含 Si, B, Se)
        self.atom_list = [6, 7, 8, 9, 15, 16, 17, 35, 53, 14, 5, 34, 0] 
        self.node_dim = len(self.atom_list)
        
        # [修改] 统一原子数上限为 48
        self.max_size = 48 + self.freedom 
        self.n_bond = 3 
        
        self.smiles_processor = SmilesPreprocessor(
            add_Hs=False, 
            kekulize=True, 
            max_atoms=48, # 确保过滤掉大于48的分子
            max_size=self.max_size
        )
        
        self.run()

    def run(self):
        print(f"开始加载数据: {self.in_path}")
        if self.max_samples is None:
            print(f"采样模式: 处理所有数据 (无限制)")
        else:
            print(f"采样限制: {self.max_samples} 条")
        
        # 强制清理旧文件
        base_dir = os.path.dirname(self.out_path)
        if not os.path.exists(base_dir): os.makedirs(base_dir)
        for ext in ['_node_features.npy', '_adj_features.npy', '_mol_sizes.npy', '_config.txt']:
            f = self.out_path + ext
            if os.path.exists(f): os.remove(f)

        all_node, all_adj, all_sizes = [], [], []
        cnt = 0
        
        if not os.path.exists(self.in_path):
            raise FileNotFoundError(f"找不到文件: {self.in_path}")

        with open(self.in_path, 'r') as fp:
            for idx, line in enumerate(fp):
                # 仅当 max_samples 被设置时才检查限制
                if self.max_samples is not None and cnt >= self.max_samples:
                    break

                smi = line.strip()
                # 跳过空行或表头
                if not smi or (idx == 0 and ('smile' in smi.lower() or smi == '0')):
                    continue

                # 处理
                node, adj, size, _ = self.smiles_processor.process(smi)
                
                if node is not None:
                    # 再次检查非法原子
                    unknown = False
                    for a in node:
                        if a != 0 and a not in self.atom_list:
                            unknown = True
                            break
                    if unknown: continue

                    all_node.append(node)
                    all_adj.append(adj[:3]) 
                    all_sizes.append(size)
                    cnt += 1
                    
                    if cnt % 5000 == 0: print(f"   已处理: {cnt} 条...")

        self.n_molecule = cnt
        print(f"处理完成！有效分子数: {cnt}")
        
        # 保存数据
        np.save(self.out_path + '_node_features.npy', np.array(all_node))
        np.save(self.out_path + '_adj_features.npy', np.array(all_adj, dtype=np.uint8))
        np.save(self.out_path + '_mol_sizes.npy', np.array(all_sizes))
        
        # 保存 Config
        config = {
            'atom_list': self.atom_list,
            'max_size': self.max_size,
            'node_dim': self.node_dim,
            'bond_dim': 4
        }
        with open(self.out_path + '_config.txt', 'w') as f:
            f.write(str(config))
        print(f"数据已保存至 {self.out_path} 前缀文件")

# ================= 3. 执行脚本 =================

input_file = 'dataset3.csv'
output_prefix = 'data_preprocessed/dataset3'

if os.path.exists(input_file):
    # 这里设置为 None，表示处理整个文件
    processor = Dataset3_Processor(input_file, output_prefix, max_samples=None)
    print("\n预处理成功！现在请运行训练代码。")
else:
    print(f"错误: 当前目录下找不到 {input_file}")

开始加载数据: dataset3.csv
采样模式: 处理所有数据 (无限制)
   已处理: 5000 条...
   已处理: 10000 条...
   已处理: 15000 条...
   已处理: 20000 条...
   已处理: 25000 条...
   已处理: 30000 条...
   已处理: 35000 条...
   已处理: 40000 条...
   已处理: 45000 条...
   已处理: 50000 条...
   已处理: 55000 条...
   已处理: 60000 条...
   已处理: 65000 条...
   已处理: 70000 条...
   已处理: 75000 条...
   已处理: 80000 条...
   已处理: 85000 条...
   已处理: 90000 条...
   已处理: 95000 条...
   已处理: 100000 条...
   已处理: 105000 条...
   已处理: 110000 条...
   已处理: 115000 条...
   已处理: 120000 条...
   已处理: 125000 条...
   已处理: 130000 条...
   已处理: 135000 条...
   已处理: 140000 条...
   已处理: 145000 条...
   已处理: 150000 条...
   已处理: 155000 条...
   已处理: 160000 条...
   已处理: 165000 条...
   已处理: 170000 条...
   已处理: 175000 条...
   已处理: 180000 条...
   已处理: 185000 条...
   已处理: 190000 条...
   已处理: 195000 条...
   已处理: 200000 条...
   已处理: 205000 条...
   已处理: 210000 条...
   已处理: 215000 条...
   已处理: 220000 条...
   已处理: 225000 条...
   已处理: 230000 条...
   已处理: 235000 条...
   已处理: 240000 条...
   已处理: 245000 条...
